In [ ]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import openpyxl
import akshare as ak
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 5000
pd.options.display.float_format = lambda x: '%.5f' % x
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='svg'
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime
import pymysql
#pwd=

In [ ]:
DB_CONFIG = {
    "host": "192.168.133.132",  # Ubuntu的IP
    "user": "root",
    "password": pwd,    
    "charset": "utf8mb4",
}
# 初始化数据库连接
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# 1. 创建数据库（如果不存在）
cursor.execute("CREATE DATABASE IF NOT EXISTS a_shares CHARACTER SET utf8mb4;")
conn.select_db("a_shares")

def get_stock_codes():
 
    try:
        df=ak.stock_zh_a_spot()[['代码','名称']].rename(columns={'代码':'code','名称':'name'})
        #上证代码（包含科创板）
        code_sh=df
        code_sh=code_sh[code_sh['code'].str.startswith('sh')]
        code_sh=code_sh[(~code_sh['name'].str.startswith('ST'))&(~code_sh['name'].str.startswith('*ST'))]
        code_sh['code']=code_sh['code'].str[2:]

        #深证代码（包含创业板）
        code_sz=df
        code_sz=code_sz[code_sz['code'].str.startswith('sz')]
        code_sz=code_sz[(~code_sz['name'].str.startswith('ST'))&(~code_sz['name'].str.startswith('*ST'))]
        code_sz['code']=code_sz['code'].str[2:]

        #北证代码
        code_bj=df
        code_bj=code_bj[code_bj['code'].str.startswith('bj')]
        code_bj=code_bj[(~code_bj['name'].str.startswith('ST'))&(~code_bj['name'].str.startswith('*ST'))]
        code_bj['code']=code_bj['code'].str[2:]

        #A股所有代码
        code=pd.concat([code_bj,code_sh,code_sz],ignore_index=True).drop_duplicates().sort_values(by='code',ascending=True).reset_index(drop=True)
        return code['code'].tolist()
    except Exception as e:
        print(f"获取股票列表失败: {e}")
        return []


In [ ]:
def fetch_and_save_stock(code):
    """爬取单只股票数据，仅当有新数据（len>0）时打印成功信息"""
    try:

        df = ak.stock_zh_a_hist(symbol=code, adjust="hfq")

        # 数据预处理
        df["日期"] = pd.to_datetime(df["日期"]).dt.date
        df.dropna(inplace=True)

        # 表名使用纯数字代码
        table_name = code

        # 判断表是否存在，不存在则创建
        cursor.execute(f"SHOW TABLES LIKE '{table_name}';")
        table_exists = cursor.fetchone() is not None

        if not table_exists:
            create_table_sql = f"""
            CREATE TABLE `{table_name}` (
                `日期` DATE NOT NULL PRIMARY KEY,
                `开盘` FLOAT,
                `收盘` FLOAT,
                `最高` FLOAT,
                `最低` FLOAT,
                `成交量` BIGINT,
                `成交额` BIGINT,
                `振幅` FLOAT,
                `涨跌幅` FLOAT,
                `涨跌额` FLOAT,
                `换手率` FLOAT
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
            """
            cursor.execute(create_table_sql)
            # 新表创建后，所有数据都是新的，无需筛选
            new_data_count = len(df)
        else:
            # 增量更新：只保留新数据
            cursor.execute(f"SELECT MAX(`日期`) FROM `{table_name}`;")
            max_date = cursor.fetchone()[0]
            if max_date:
                df = df[df["日期"] > max_date]
            new_data_count = len(df)
            if new_data_count == 0:
                # 无新数据，直接跳过（不打印成功信息）
                return

        # 批量插入新数据
        insert_sql = f"""
        INSERT IGNORE INTO `{table_name}` 
        (日期, 开盘, 收盘, 最高, 最低, 成交量, 成交额, 振幅, 涨跌幅, 涨跌额, 换手率)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        data = [tuple(row) for row in df[["日期", "开盘", "收盘", "最高", "最低", 
                                         "成交量", "成交额", "振幅", "涨跌幅", "涨跌额", "换手率"]].values]
        cursor.executemany(insert_sql, data)
        conn.commit()

        # 仅当有新数据（len>0）时打印成功信息
        if new_data_count > 0:
            print(f"股票 {code} 处理成功，新增 {new_data_count} 条数据")

    except Exception as e:
        print(f"股票 {code} 处理失败: {str(e)}")
        conn.rollback()

In [ ]:
stock_codes = get_stock_codes()
for code in stock_codes:
    fetch_and_save_stock(code)

cursor.close()
conn.close()